# News Summarization

In [1]:
import pandas as pd
import numpy as np
import datasets
from datasets import Dataset, DatasetDict
import evaluate
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline, DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

/opt/homebrew/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/opt/homebrew/lib/python3.10/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c1017RegisterOperatorsD1Ev
  Referenced from: <9A4710B9-0DA3-36BB-9129-645F282E64B2> /opt/homebrew/lib/python3.10/site-packages/torchvision/image.so
  Expected in:     <7856C0E5-3D52-39C7-8515-71217150BD2E> /opt/homebrew/lib/python3.10/site-packages/torch/lib/libtorch_cpu.dylib'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
# Loading the testing dataset
df_test = pd.read_csv('test.csv')
df_test = df_test.drop(columns=['id'])

In [3]:
model_name = "bart-news"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Create a summarization pipeline
summarizer = pipeline("summarization", model=model, tokenizer=tokenizer)

# Extract model parameters
def get_model_parameters(model):
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    return total_params, trainable_params

total_params, trainable_params = get_model_parameters(model)
print(f"Total Parameters: {total_params}")
print(f"Trainable Parameters: {trainable_params}")


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


Total Parameters: 406290432
Trainable Parameters: 406290432


In [5]:
rouge = evaluate.load("rouge")
bleu = evaluate.load("bleu")

rouge_1 = []
rouge_2 = []
rouge_L = []
bleu_val = []

# Initialize the summarizer once outside the loop
summarizer = pipeline("summarization", model=model, tokenizer=tokenizer)
for i in range(1000):  

    try:  
        # Generate the summary
        summary = summarizer(df_test['article'].iloc[i])
        # Print the summary
        generated_summary = summary[0]['summary_text']
        
        # Compute ROUGE score
        rouge_score = rouge.compute(predictions=[generated_summary], references=[df_test['highlights'].iloc[i]], use_aggregator=True)
        rouge_1.append(rouge_score["rouge1"])
        rouge_2.append(rouge_score["rouge2"])
        rouge_L.append(rouge_score["rougeL"])

        # Compute BLEU score
        bleu_score = bleu.compute(predictions=[generated_summary], references=[[df_test['highlights'].iloc[i]]])
        bleu_val.append(bleu_score["bleu"])
    except:
        continue

print(f"ROUGE 1 Score : {np.average(rouge_1)}")
print(f"ROUGE 2 Score : {np.average(rouge_2)}")
print(f"ROUGE L Score : {np.average(rouge_L)}")
print(f"BLEU Score : {np.average(bleu_val)}")

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


0
ROUGE 1 Score : 0.5084745762711865
ROUGE 2 Score : 0.3620689655172414
ROUGE L Score : 0.44067796610169496
BLEU Score : 0.3520696440183301


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [14]:
# Initialize the summarizer once outside the loop
summarizer = pipeline("summarization", model=model, tokenizer=tokenizer)
summary = summarizer(df_test['article'].iloc[0])
generated_summary = summary[0]['summary_text']
        
print(df_test['article'].iloc[0])
print(df_test['highlights'].iloc[0])
print(generated_summary)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


By . Associated Press . PUBLISHED: . 14:11 EST, 25 October 2013 . | . UPDATED: . 15:36 EST, 25 October 2013 . The bishop of the Fargo Catholic Diocese in North Dakota has exposed potentially hundreds of church members in Fargo, Grand Forks and Jamestown to the hepatitis A virus in late September and early October. The state Health Department has issued an advisory of exposure for anyone who attended five churches and took communion. Bishop John Folda (pictured) of the Fargo Catholic Diocese in North Dakota has exposed potentially hundreds of church members in Fargo, Grand Forks and Jamestown to the hepatitis A . State Immunization Program Manager Molly Howell says the risk is low, but officials feel it's important to alert people to the possible exposure. The diocese announced on Monday that Bishop John Folda is taking time off after being diagnosed with hepatitis A. The diocese says he contracted the infection through contaminated food while attending a conference for newly ordained b